# Introduction to Visualization In Action - Part 3


## What will we do in this notebook: 

So far we've seen matplotlib's functionality as a plotting package. Now we are going to explore more of the Python plotting ecosytem. Will be using elevation data that represents Mars' surface in this lab to demonstrate various additional plotting. 

-

-

-

-

-

## Section 1. Read in Data 

The following data files are in a new format. These are "sav" files from a commonly used programming language called IDL. It is likely that some of your project will need to use these IDL specific files. These data files are specifically from the MAVEN suite of tools which can be found [online](https://lasp.colorado.edu/maven/sdc/public/pages/software.html).

Luckily there is a function in Python that let's us read and use these files called readsav.

In [ ]:
from scipy.io import readsav #for data reading

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
exampleRead = readsav('./Data/MarsTopo/lat.sav')

In [ ]:
print(exampleRead)

In [ ]:
print(type(exampleRead))

### Discussion: Why do you think it's useful to print the 'type' as well like we just did? 

-

-

-

-

-

## Section 2. Read in the Data 

Let's make things a littler easier on us by defining a function to read the data we need. 

In [ ]:

def readTopo():
    '''
    Reads topo data from stored .sav files.
    
    Resolution currenly at 0.2 degrees lon x lat.
    '''

    #a sav file is an IDL specific file...common in space physics
    lat = readsav('./Data/MarsTopo/lat.sav')

    lat = lat['lat']

    lon = readsav('./Data/MarsTopo/elon.sav')

    lon = lon['elon']

    alt = readsav('./Data/MarsTopo/alt_topo_ell.sav')

    alt = alt['alt_topo_ell']*-1 #reversing scale 
    
    return(lon, lat, alt)
    

In [ ]:
#using our function to get our data

lon, lat, alt = readTopo()

In [ ]:
#let's take a look at these datasets

print(type(lat))
print(type(lon))
print(type(alt))

print(lat.shape)
print(lon.shape)
print(alt.shape)

This is a "gridded" data product, each altitude value corresponds to a specific latitude and longitude as you can see by the dimensions printed above.

There are a few ways to look at data like this with the basic matplotlib like we've seen before, for example with pcolormesh. 

In [ ]:
plt.pcolormesh(alt, cmap = 'YlOrRd') #you can find more color maps at 
                                     #https://matplotlib.org/stable/tutorials/colors/colormaps.html

### Discussion: Let's pause here. What might be some potential issues (accuracy and aesthetics both) with the above plot? 

-

-

-

-

-

## Section 3. Let's explore some other useful packages

Seaborn is a generally useful plotting package that allows you to do statistical visualizations and wraps much of matplotlib's functionality. Find out more [online](https://seaborn.pydata.org/index.html). Seaborn often combines statistical analyses with plotting so make sure to read the documentation about this package when using it to understand what exactly is being plotted. 


In [ ]:
### First we need to install seaborn as it is not included in our default library on this platform 
!pip install seaborn

In [ ]:
### Now it can be imported

import seaborn as sns

sns.heatmap(alt)



You can see we have similar issues with matplotlib as with seaborn in terms of gridding and projections of our dataset. Let's try some packages specifically made for geographic data.

We will use cartopy next. Cartopy is a way to visualize geographic data. This often include functions regarding projections that intersect with matplotlib. Find out more about cartopy [online](https://scitools.org.uk/cartopy/docs/latest/). 

In [ ]:
import cartopy.crs as ccrs

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree()) #project onto flat surface
ax.pcolormesh(lon, lat, alt, transform = ccrs.PlateCarree(), cmap = 'YlOrRd') #use transformation from flat surface


In [ ]:

ax = plt.axes(projection=ccrs.Orthographic()) #project onto globe
ax.pcolormesh(lon, lat, alt, transform = ccrs.PlateCarree(), cmap = 'YlOrRd') #use transformation from flat surface

#notice it's a little tricky in terms of projection syntax

### Discussion: What do you notice that is different from these projections and the previous ones in seaborn and matplotlib?

-

-

-

-

-

# Section 4. Interactive Plotting

Cartopy is often what you will use if doing projections. The syntax usually "adds" to matplotlib's by incuding more keywords or other transformation as we just saw. However, when you want to explore a more global view, it can be helpful to have 3D visualization potential. We are going to now move into creating an interactive globe of Mars. For this we will use a new package, plotly (find more information [online](https://plotly.com/)) which is designed for interactive visualizations. 

In [ ]:
# as with seaborn this package doesn't exist on our hub so we have to install it

!pip install plotly

Unlike cartopy, seaborn, and matplotlib, plotly's syntax begins to diverge significantly from what you've seen before. There are quite a few ways to interact with plotly. Let's start with something basic before going into the Mars' applications. 

In [ ]:
#this is one of the basic plotly examples of a scatter plot

import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=[1, 2, 3, 4], y=[10, 11, 12, 13], mode='markers', marker=dict(size=[40, 60, 80, 100],
                color=[0, 1, 2, 3])))

fig.show()

### Discussion: Explore the previous plot. What questions do you have about some of the differences between plotly and other plotting packages?

-

-

-

-

-

-

## Section 5. Global Interactive Plotting

Unfortunately, the projections that we could use in cartopy are not supported in plotly so we need to define a few functions ourselves ([reference source](https://nordicesmhub.github.io/deep_python/)).

In [ ]:
def degree2radians(degree):
    #convert degrees to radians
    return degree*np.pi/180

def mapping_map_to_sphere(lon, lat, radius=1):
    #this function maps the points of coords (lon, lat) to points onto the  sphere of radius radius
    
    lon=np.array(lon, dtype=np.float64)
    lat=np.array(lat, dtype=np.float64)
    lon=degree2radians(lon)
    lat=degree2radians(lat)
    xs=radius*np.cos(lon)*np.cos(lat)
    ys=radius*np.sin(lon)*np.cos(lat)
    zs=radius*np.sin(lat)
    return xs, ys, zs

In [ ]:
#we are going to take every second data point so we don't have to wait forever
#for these plots to render, you can bump this up if you want, just remove the [::2]
lon = lon[::2]
lat = lat[::2]
alt = alt[::2, ::2]

#we now make a grid of the data
clon, clat = np.meshgrid(lon, lat)

#see how these are now all the same dimensions?
print(clon.shape, clat.shape, alt.shape)

In [ ]:

#actual Mars in 3D...if you want 'flat' mars change radius to 1
#transforms to spherical
X_elev, Y_elev, Z_elev = mapping_map_to_sphere(clon, clat, radius = (1000+(alt)))


#create elevation surface
elevationSurf = go.Surface(x=X_elev, y=Y_elev, z=Z_elev, colorscale='Brwnyl_r',
                               surfacecolor = (alt))

We still haven't plotted anything yet. This requires we have to now make a figure.

In [ ]:
#make figure
fig = go.Figure([elevationSurf])


In [ ]:
#show figure
fig.show()

### Discussion: What do we think could improve this figure? How can we refine this?

-

-

-

-

-

-

-

## Section 6. Refining Mini Mars

Let's remove some of the clutter and add some labels.

In [ ]:
lon, lat, alt = readTopo() # re-read the data in, this time let's not subset the data

#we now make a grid of the data
clon, clat = np.meshgrid(lon, lat)


#making a new background
noaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False,
            ticks='', title='', zeroline=False)


layout = go.Layout(title="Mars Elevation",
                    scene = dict(
                    xaxis = noaxis,
                    yaxis = noaxis,
                    zaxis = noaxis,
                    camera=dict(eye=dict(x=-1.5, y=0, z=0.2))))


In [ ]:
#map to sphere
X_elev, Y_elev, Z_elev = mapping_map_to_sphere(clon, clat, radius = (1000+(alt)))


#create elevation surface
elevationSurf = go.Surface(x=X_elev, y=Y_elev, z=Z_elev, colorscale='Brwnyl_r',
                               surfacecolor = (alt), hoverinfo='skip',
                               colorbar = dict(title = dict(text = 'Elevation (km) \n', side = 'top')))

In [ ]:
#Instead of showing the figure we are going to save it and see how it looks as a .html file

fig = go.Figure([elevationSurf], layout=layout)


In [ ]:
#this cell will take some time to run
fig.write_html('MarsElevationExample.html', include_plotlyjs="cdn")

### Activity: Take some time to rerun this section with different colorscales (find them [here](https://plotly.com/python/builtin-colorscales/). What else might you change in this figure?